In [35]:
# Import necessary modules
from tests.MOF_builder.functions.ciftemplate2graph import ct2g,node_vecs
import networkx as nx
import numpy as np
import os
import veloxchem as vlx
import re
from numpy.linalg import norm
from tests.MOF_builder.functions.bbcif_properties import bb2array,X_vecs,selected_type_vecs
from tests.MOF_builder.functions.place_bbs import superimpose,mag_superimpose

In [36]:
from src.MOF_builder.functions.frag_recognizer import process_linker_molecule
linker_file = 'ndi.xyz'
molecule = vlx.Molecule.read_xyz_file(linker_file)
linker_topic =2
center_frag_nodes_num,center_Xs,single_frag_nodes_num,frag_Xs= process_linker_molecule(molecule,linker_topic)

center is two points
[9, 14]
ditopic linker: center are two points
find connected X in edge:   24
find connected X in edge:   39
edges/diedge.cif is writen
center_frag: 46 [18, 33]


In [37]:
# Display edges of the graph


'''
Er1     Er     0.8416   0.75   0.1586
Er2     Er     0.6584   0.75   0.3414
Er3     Er     0.8416   0.25   0.8414
Er4     Er     0.6584   0.25   0.6586
V5     V     0.5   0.5   0.5
V6     V     0.5   0.0   0.5
Er7     Er     0.1584   0.25   0.8414
Er8     Er     0.3416   0.25   0.6586
Er9     Er     0.1584   0.75   0.1586
Er10     Er     0.3416   0.75   0.3414
V11     V     0.0   0.0   0.0
V12     V     0.0   0.5   0.0
loop_
_geom_bond_atom_site_label_1
_geom_bond_atom_site_label_2
_geom_bond_distance
_geom_bond_site_symmetry_2
_ccdc_geom_bond_type
Er1     Er2    10.0   .     S
Er1     V11    10.0   1_665     S
Er1     V12    10.0   1_655     S
Er2     V5    10.0   .     S
Er2     V6    10.0   1_565     S
Er3     Er4    10.0   .     S
Er3     V12    10.0   1_656     S
Er3     V11    10.0   1_656     S
Er4     V5    10.0   .     S
Er4     V6    10.0   .     S
V5     V6    10.0   .     S
V5     V6    10.0   1_565     S
V5     Er8    10.0   .     S
V5     Er10    10.0   .     S
V6     V5    10.0   1_545     S
V6     Er8    10.0   .     S
V6     Er2    10.0   1_545     S
V6     Er10    10.0   1_545     S
Er7     Er8    10.0   .     S
Er7     V11    10.0   1_556     S
Er7     V12    10.0   1_556     S
Er9     Er10    10.0   .     S
Er9     V11    10.0   1_565     S
Er9     V12    10.0   .     S
Er10     V6    10.0   1_565     S
V11     V12    10.0   1_545     S
V11     Er1    10.0   1_445     S
V11     V12    10.0   .     S
V11     Er3    10.0   1_454     S
V11     Er9    10.0   1_545     S
V11     Er7    10.0   1_554     S
V12     V11    10.0   1_565     S
V12     Er1    10.0   1_455     S
V12     Er3    10.0   1_454     S
V12     Er7    10.0   1_554     S
'''

'\nEr1     Er     0.8416   0.75   0.1586\nEr2     Er     0.6584   0.75   0.3414\nEr3     Er     0.8416   0.25   0.8414\nEr4     Er     0.6584   0.25   0.6586\nV5     V     0.5   0.5   0.5\nV6     V     0.5   0.0   0.5\nEr7     Er     0.1584   0.25   0.8414\nEr8     Er     0.3416   0.25   0.6586\nEr9     Er     0.1584   0.75   0.1586\nEr10     Er     0.3416   0.75   0.3414\nV11     V     0.0   0.0   0.0\nV12     V     0.0   0.5   0.0\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_geom_bond_site_symmetry_2\n_ccdc_geom_bond_type\nEr1     Er2    10.0   .     S\nEr1     V11    10.0   1_665     S\nEr1     V12    10.0   1_655     S\nEr2     V5    10.0   .     S\nEr2     V6    10.0   1_565     S\nEr3     Er4    10.0   .     S\nEr3     V12    10.0   1_656     S\nEr3     V11    10.0   1_656     S\nEr4     V5    10.0   .     S\nEr4     V6    10.0   .     S\nV5     V6    10.0   .     S\nV5     V6    10.0   1_565     S\nV5     Er8    10.0   .     S\nV5    

In [38]:
#pillar stack structure:
#regarding chain_node cif file:
    #find the pillar direction vector which is defined by the Al-Al vectors in chain_node cif file 
    #find the pair of x vectors, which are vertical to the pillar direction vector in chain_node cif file, could be more than one pairs
    #find the Al-Al length, read from the cif file
#regarding the linker cif file:
    #find the length of the linker which should be the maximum length of X vectors in the linker cif file

#regarding the template cif file:
    #find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file, 
    #find the edge vector in the template cif file, which should be Er-Er edge
    #find the length of the edge vector in the template cif file
    #fix one direction of the box, which should be parallel to the pilalar direction

#the unit_cell of the template cif file should be scaled to fit the length of the linker, and the box_vector which along pillar direction should be fixed
#when we want to place the node in the box, we should use a pair of x vectors which targets the connected V through the edge vector in the template cif file and the pillar direction vector, we use superimpose function to rotate and place the node at the right position
#when we want to place the edge in the box, we should consider all of the x vectors placed in box and find the connected connected x vectors in two sidde of the Er-Er edge, then we use superimpose function to rotate and place the edge at the right position 
from tests.MOF_builder.functions.bbcif_properties import bb2array,selected_type_vecs,calc_edge_len
from tests.MOF_builder.functions.place_bbs import superimpose

chain_node_cif = '21Alchain.cif'
template_cif = 'chain_rna.cif'
linker_cif = 'edges/diedge.cif'
#chain node
#Al_vecs = selected_type_vecs(chain_node_cif,'.', 'Al',False)
#chain_node_vecs = bb2array(chain_node_cif,'.')
#chain_node_coords = np.asarray([i[1] for i in chain_node_vecs])
#metal_cvec = Al_vecs[0]-Al_vecs[1]
#node_pillar_cvec = metal_cvec/norm(metal_cvec) 
#node_x_vecs = selected_type_vecs(chain_node_cif,'.','X',False)


##find the x vectors which are vertical to the pillar direction
#xx_pairs = []
#for i in range(0,len(x_vecs)):
#    for j in range(i+1,len(x_vecs)):
#        xx_vec = x_vecs[i]-x_vecs[j]
#        xx_vec = xx_vec/norm(xx_vec)
#        if np.dot(xx_vec,metal_cvec) < 1e-3:
#            xx_pairs.append((i,j))
#print(xx_pairs)


#linker
linker_x_vecs = selected_type_vecs(linker_cif,'.','X',False)
#ditopic linker only has two x vectors
linker_length = calc_edge_len(linker_cif,'.') #length of the edge should be x-x length in linker cif file, unit angstrom
print(linker_length,'linker_length')

###template
##gen = ct2g('chain_rna.cif', 'tests/templates')
##net = next(gen)
##TG, start, unit_cell, TVT, TET, TNAME, a, b, c, ang_alpha, ang_beta, ang_gamma, max_le, catenation = net
##
##def updateTG2NG(TG):
##    #ditopic linker should be connected to two V nodes directly 
##    #find path between two nodes which starts from V node to V node passing through Er nodes
##    Vnodes = [n for n in TG.nodes() if n.startswith('V')]
##    Ernodes = [n for n in TG.nodes() if n.startswith('Er')]
##
##    valid_paths = []
##    Vnodes = [n for n in TG.nodes() if n.startswith('V')]
##    for i in range(0, len(Vnodes)):
##        for j in range(i+1, len(Vnodes)):
##            source=Vnodes[i]
##            target=Vnodes[j]
##            path = nx.shortest_path(TG, source,target)
##            if len(path) > 2:
##                #if only two V nodes are there in the path passed Er, then it is a valid path
##                if all([n.startswith('Er') for n in path[1:-1]]):
##                    valid_paths.append(path)
##
##    NG = nx.Graph()
##    edges_info=TG.edges(data=True)
##    count = 0
##    le=0
##    pillars_cvecs = []
##    pillars_fvecs = []
##    edge_cvecs = []
##    edge_fvecs = []
##    for edge in edges_info:
##        #node1,node2=edge[0],edge[1]
##        edge_type=edge[2]['type']
##        edge_label=edge[2]['label']
##        edge_ccoord=edge[2]['ccoords']
##        edge_fcoord=edge[2]['fcoords']
##        edge_length=edge[2]['length']
##        edge_index=edge[2]['index']
##        edge_pd = edge[2]['pd']
##        node1,node2 = edge_pd
##    
##        if edge_type[0]==edge_type[1]:
##            #V-V find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
##            if edge_type[0]=='V':# chain of metal nodes
##                #print(node1,node2,edge_type,edge_fcoord,edge_length,edge_index,edge_pd)
##                #name = node1+node2+'_'+str(count)
##                name = int(count)
##                pillars_cvecs.append(TG.nodes[node1]['ccoords']-TG.nodes[node2]['ccoords'])
##                pillars_fvecs.append(TG.nodes[node1]['fcoords']-TG.nodes[node2]['fcoords'])
##                NG.add_node(name, label = (node1,node2),type=edge_type, index=edge_index, ccoords=edge_ccoord, fcoords=edge_fcoord, cn=[], cifname=[])
##                count+=1
##            #Er-Er find the edge vector in the template cif file, which should be Er-Er edge
##            else:
##                le = max(edge_length,le) #find the length of the edge vector in the template cif file
##                edge_cvecs.append(TG.nodes[node1]['ccoords']-TG.nodes[node2]['ccoords'])
##                edge_fvecs.append(TG.nodes[node1]['fcoords']-TG.nodes[node2]['fcoords'])
##                pass
##                #edge direction 
##                #print(node1,node2,edge_type,edge_fcoord,edge_length,edge_index,edge_pd)
##                #get evecs
##
##        else:
##            #ignore Er-V
##            #print(node1,node2,edge_type)
##            pass
##
##    #in NG any VV node has 'V5' should be connected to antoher VV node with 'V11' and 'V12'
##    for path in valid_paths:
##        source = path[0]
##        target = path[-1]
##        #in NG any VV node has source should be connected to antoher VV node with target
##        vv_source_nodes = [node for node in NG.nodes() if source in NG.nodes[node]['label']]
##        vv_target_nodes = [node for node in NG.nodes() if target in NG.nodes[node]['label']]
##        #connect any source node to any target node
##
##
##    #find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
##    count = 1
##    edges_cvecs=[]
##    edges_fvecs=[]
##    for vv_source_node in vv_source_nodes:
##        for vv_target_node in vv_target_nodes:
##            fcoord = (NG.nodes[vv_source_node]['fcoords'],NG.nodes[vv_target_node]['fcoords'])
##            ccoord = (NG.nodes[vv_source_node]['ccoords'],NG.nodes[vv_target_node]['ccoords'])
##            edges_cvecs.append(NG.nodes[vv_target_node]['ccoords']- NG.nodes[vv_source_node]['ccoords'])
##            edges_fvecs.append(NG.nodes[vv_target_node]['fcoords']- NG.nodes[vv_source_node]['fcoords'])
##            lbl = [0,0,0]
##            NG.add_edge(vv_source_node,vv_target_node,key=(count,lbl[0],lbl[1],lbl[2]), label=lbl , length=le, fcoords=fcoord, ccoords=ccoord, index=count, pd=(vv_source_node,vv_target_node))
##            count+=1
##    return NG,pillars_cvecs
####
####NG,pillars_cvecs = updateTG2NG(TG)
#####fix one direction of the box, which should be parallel to the pilalar direction
####
####
####
#####the unit_cell of the template cif file should be scaled to fit the length of the linker, and the box_vector which along pillar direction should be fixed
####
####
#####find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
####
##
###find the pillar direction which should be along the pure V-V direction (no Er node between) in the template cif file
##


15.754259612561926 linker_length


In [39]:
from tests.MOF_builder.functions._readcif import extract_type_atoms_array_in_primitive_cell,read_cif,extract_atoms_xyz_from_lines,extract_quote_lines,extract_symmetry_operation_from_lines,apply_sym_operator
import numpy as np

target_type = 'Al'
cell_info, array_atom, array_target_atoms=extract_type_atoms_array_in_primitive_cell(chain_node_cif, target_type)
_,_, node_x_vecs=extract_type_atoms_array_in_primitive_cell(chain_node_cif, 'X')

cell_info, symmetry_sector, atom_site_sector = read_cif(chain_node_cif)
array_atom, array_xyz = extract_atoms_xyz_from_lines(atom_site_sector)
node_com = np.mean(array_target_atoms, axis=0)
chain_node_fcoords = array_xyz - node_com

metal_cvec = array_target_atoms[0]-array_target_atoms[1]
node_pillar_cvec = metal_cvec/norm(metal_cvec) 
node_x_vecs = node_x_vecs - node_com


apply 7 symmetry operation
0  no symmetry operation
apply 7 symmetry operation
0  no symmetry operation


In [40]:
import numpy as np
import networkx as nx
from tests.MOF_builder.functions._readcif import extract_type_atoms_array_in_primitive_cell
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster

cif_file ='MIL53templatecif.cif'
target_type = 'YY'
cluster_distance_threshhold = 0.1

# use cell_info to generate the matrix for the unit cell to get cartesian coordinates
def make_supercell_3x3x3(array_xyz):
    array_x1 = array_xyz + np.array([1, 0, 0])
    array_x2 = array_xyz + np.array([-1, 0, 0])
    array_y1 = array_xyz + np.array([0, 1, 0])
    array_y2 = array_xyz + np.array([0, -1, 0])
    array_x1_y1 = array_xyz + np.array([1, 1, 0])
    array_x1_y2 = array_xyz + np.array([1, -1, 0])
    array_x2_y1 = array_xyz + np.array([-1, 1, 0])
    array_x2_y2 = array_xyz + np.array([-1, -1, 0])
    
    layer_3x3 = np.vstack((
        array_xyz,
        array_x1,
        array_x2,
        array_y1,
        array_y2,
        array_x1_y1,
        array_x1_y2,
        array_x2_y1,
        array_x2_y2
    ))
    
    layer_3x3_z1 = layer_3x3 + np.array([0, 0, 1])
    layer_3x3_z2 = layer_3x3 + np.array([0, 0, -1])
    
    supercell_3x3x3 = np.vstack((
        layer_3x3,
        layer_3x3_z1,
        layer_3x3_z2
    ))
    
    return supercell_3x3x3

def extract_unit_cell(cell_info):
    pi = np.pi
    aL, bL, cL, alpha, beta, gamma = cell_info
    aL,bL,cL,alpha,beta,gamma = list(map(float, (aL,bL,cL,alpha,beta,gamma)))
    ax = aL
    ay = 0.0
    az = 0.0
    bx = bL * np.cos(gamma * pi / 180.0)
    by = bL * np.sin(gamma * pi / 180.0)
    bz = 0.0
    cx = cL * np.cos(beta * pi / 180.0)
    cy = (cL * bL * np.cos(alpha * pi /180.0) - bx * cx) / by
    cz = (cL ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
    unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
    return unit_cell
# a function, given an array of atoms, use distance to find the clusters of atoms, and return the center of the cluster, we can set a distance thereshold to define the cluster
def find_cluster_center(array_atom):
    array_atom = np.array(array_atom,dtype=float)
    center = np.mean(array_atom,axis=0)
    return center

def clust_analysis_points(array_atom,distance_threshhold):
    #use scipy.cluster distance, to cluster the points
    #find the distance matrix
    dist = pdist(array_atom)
    #find the linkage matrix
    Z = linkage(dist, 'ward')
    #find the cluster
    cluster = fcluster(Z, distance_threshhold, criterion='distance')
    #find the center of the cluster
    cluster_center = []
    for i in range(1,max(cluster)+1):
        cluster_center.append(find_cluster_center(array_atom[cluster==i]))
    return cluster_center

def extract_cluster_center_from_templatecif(cif_file, target_type,cluster_distance_threshhold):
    cell_info, array_atom, array_target_atoms =extract_type_atoms_array_in_primitive_cell(cif_file, target_type)
    unit_cell = extract_unit_cell(cell_info)
    unit_cell = np.round(unit_cell,3)
    metal333 = make_supercell_3x3x3(array_target_atoms)
    metal333 = np.vstack(metal333)
    #cluster analysis in cartesian coordinates
    array_metal_ccords = np.dot(unit_cell,metal333.T).T
    cluster_centers_ccoords=clust_analysis_points(array_metal_ccords,cluster_distance_threshhold)
    cluster_centers_ccoords = np.vstack(cluster_centers_ccoords)
    #cluster_centers should return fractional coordinates
    cluster_centers_fcoords = np.dot(np.linalg.inv(unit_cell),cluster_centers_ccoords.T).T
    #filter cluster centers which is inside the unit cell, boundary condition is [-0.01,1.01]
    cluster_centers_fcoords = [c for c in cluster_centers_fcoords if all([i>=-0.01 and i<=1.01 for i in c])]
    cluster_centers_fcoords = np.mod(cluster_centers_fcoords,1)
    cluster_centers_fcoords = np.round(cluster_centers_fcoords,3)
    cluster_centers_fcoords = np.unique(cluster_centers_fcoords,axis=0)
    return cluster_centers_fcoords,cell_info,unit_cell

In [41]:
vvnode,cell_info,unit_cell = extract_cluster_center_from_templatecif(cif_file, 'YY',1)
eenode,_,_ = extract_cluster_center_from_templatecif(cif_file, 'XX',1)

#loop over super333xxnode and super333yynode to find the pair of x node in unicell which pass through the yynode
vvnode333 = make_supercell_3x3x3(vvnode)
eenode333 = make_supercell_3x3x3(eenode)
#find pair of x, pass y, which means y is the edge center between two x 

#for each y, find nearest x in xxnode333, then check if the center of the pair of x is around y, if yes, the it is valid pair of x
def check_inside_unit_cell(point):
    return all([i>=-0.01 and i<=1.01 for i in point])

#check if after np.mod, the fcoords is the same as before
def check_moded_fcoords(point):
    x,y,z = point[0],point[1],point[2]
    if np.mod(x,1)!=x:
        return False
    if np.mod(y,1)!=y:
        return False  
    if np.mod(z,1)!=z:
        return False
    return True

def find_pair_v_e(vvnode333, eenode333):
    G = nx.Graph()
    pair_ve = []
    for e in eenode333:
        dist = np.linalg.norm(vvnode333 - e, axis=1)
        # find two v which are nearest to e, and at least one v is in [0,1] unit cell
        v1 = vvnode333[np.argmin(dist)]
        v1_idx = np.argmin(dist)
        dist[np.argmin(dist)] = 1000
        v2 = vvnode333[np.argmin(dist)]
        v2_idx = np.argmin(dist)
        # find the center of the pair of v
        center = (v1 + v2) / 2
        # check if there is a v in [0,1] unit cell
        if check_inside_unit_cell(v1) or check_inside_unit_cell(v2):
            # check if the center of the pair of v is around e
            if np.linalg.norm(center - e) < 1e-3:
                G.add_node(v1_idx, fcoords=v1)
                G.add_node(v2_idx, fcoords=v2)
                G.add_edge(v1_idx, v2_idx, fcoords=(v1, v2),fc_center=e),
                pair_ve.append((v1, v2, e))
    return pair_ve, len(pair_ve), G

#add ccoords to the the nodes in the graph
def add_ccoords(G,unit_cell):
    for n in G.nodes():
        G.nodes[n]['ccoords'] = np.dot(unit_cell,G.nodes[n]['fcoords'])
    return G

def set_DV_V(G):
    for n in G.nodes():
        if G.degree(n) == max(dict(G.degree()).values()):
            #G.nodes[n]['type'] = 'V'
            #check if the moded ccoords is in the unit cell
            if check_moded_fcoords(G.nodes[n]['fcoords']):
                G.nodes[n]['type'] = 'V'
            else:
                G.nodes[n]['type'] = 'DV'
        else:
            G.nodes[n]['type'] = 'DV'
    return G

#check e in G, find e in unit_cell, use np.mod to filter the e in unit_cell, set the valid e with E type, others are DE type
def set_DE_E(G):
    all_e =[]
    for e in G.edges():
        all_e.append(G.edges[e]['fc_center'])
    all_e = np.vstack(all_e)
    all_e = np.mod(all_e,1)
    unique_e = np.unique(all_e,axis=0)
    for e in G.edges():
        if np.any(np.all(np.isclose(G.edges[e]['fc_center'],unique_e),axis=1)):
            G.edges[e]['type'] = 'E'
        else:
            G.edges[e]['type'] = 'DE'
    return G



pair_vv_e,_,G=find_pair_v_e(vvnode333,eenode333)
G = add_ccoords(G,unit_cell)
G = set_DV_V(G)
G = set_DE_E(G)


apply 16 symmetry operation
16  symmetry operation
apply 16 symmetry operation
16  symmetry operation


In [42]:
#check connectivity of each node in G
def check_connectivity(G):
    for n in G.nodes():
        print(n,G.nodes[n],G.degree(n))
check_connectivity(G)

#check edge information in G
def check_edge(G):
    for e in G.edges():
        print(e,G.edges[e])
check_edge(G)


1 {'fcoords': array([0.  , 0.75, 0.25]), 'ccoords': array([ 0.     , 12.66975,  3.148  ]), 'type': 'V'} 2
72 {'fcoords': array([ 0.  ,  0.25, -0.25]), 'ccoords': array([ 0.     ,  4.22325, -3.148  ]), 'type': 'DV'} 1
2 {'fcoords': array([0.5 , 0.25, 0.75]), 'ccoords': array([3.3115 , 4.22325, 9.444  ]), 'type': 'V'} 2
19 {'fcoords': array([ 0.5 , -0.25,  0.25]), 'ccoords': array([ 3.3115 , -4.22325,  3.148  ]), 'type': 'DV'} 1
3 {'fcoords': array([0.5 , 0.75, 0.25]), 'ccoords': array([ 3.3115 , 12.66975,  3.148  ]), 'type': 'V'} 2
5 {'fcoords': array([1.  , 0.75, 0.25]), 'ccoords': array([ 6.623  , 12.66975,  3.148  ]), 'type': 'DV'} 2
76 {'fcoords': array([ 1.  ,  0.25, -0.25]), 'ccoords': array([ 6.623  ,  4.22325, -3.148  ]), 'type': 'DV'} 1
84 {'fcoords': array([ 0.  ,  1.25, -0.25]), 'ccoords': array([ 0.     , 21.11625, -3.148  ]), 'type': 'DV'} 1
14 {'fcoords': array([0.5 , 1.25, 0.75]), 'ccoords': array([ 3.3115 , 21.11625,  9.444  ]), 'type': 'DV'} 1
92 {'fcoords': array([ 1. 

In [43]:
#firstly, check if all V nodes have highest connectivity
#secondly, sort all DV nodes by connectivity
def sort_nodes_by_type_connectivity(G):
    Vnodes = [n for n in G.nodes() if G.nodes[n]['type']=='V']
    DVnodes = [n for n in G.nodes() if G.nodes[n]['type']=='DV']
    Vnodes = sorted(Vnodes,key=lambda x: G.degree(x),reverse=True)
    DVnodes = sorted(DVnodes,key=lambda x: G.degree(x),reverse=True)
    return Vnodes+DVnodes
sorted_nodes = sort_nodes_by_type_connectivity(G)


In [44]:
#fix one direction of the box, which should be parallel to the pilalar direction

#rotate the node to pillar direction and put all nodes into the cartesian coordinate 
def check_edge_inunitcell(e):
    if 'DV' in G.nodes[e[0]]['type'] or 'DV' in G.nodes[e[1]]['type']:
        return False
    return True

def find_and_sort_edges_bynodeconnectivity(graph, sorted_nodes):
    all_edges = list(graph.edges())

    sorted_edges = []
    #add unit_cell edge first

    ei = 0
    while ei < len(all_edges):
            e = all_edges[ei]
            if check_edge_inunitcell(e):
                sorted_edges.append(e)
                all_edges.pop(ei)
            ei += 1
    #sort edge by sorted_nodes
    for n in sorted_nodes:
        ei = 0
        while ei < len(all_edges):
            e = all_edges[ei]
            if n in e:
                if n ==e[0]:
                    sorted_edges.append(e)
                else:
                    sorted_edges.append((int(e[1]),int(e[0])))
                all_edges.pop(ei)
            else:
                ei += 1

    return sorted_edges    
    
sorted_edges = find_and_sort_edges_bynodeconnectivity(G,sorted_nodes)




In [45]:
#debug
###save the sorted nodes ccoords as xyz
##def save_node_xyz(sorted_nodes,unit_cell,G):
##    with open('sorted_nodes.xyz','w') as f:
##        f.write(str(len(sorted_nodes))+'\n')
##        f.write('sorted_nodes\n')
##        for n in sorted_nodes:
##            ccoords = np.dot(unit_cell,G.nodes[n]['fcoords']).tolist()
##            f.write('V'+str(sorted_nodes.index(n))+'    '+' '.join(map(str,ccoords))+'\n')
##save_node_xyz(sorted_nodes,unit_cell,G)

In [46]:
#loop node in G, for each node, find the nearest node, and calculte the node_node vector, check if all of the pair vector are parallel
from numpy.linalg import norm
def check_if_pillarstack(G):
    node_node_vec = []
    for n in G.nodes():
        if 'DV' in G.nodes[n]['type']:
            continue
        dist = []
        for nn in G.nodes():
            dist.append(np.linalg.norm(G.nodes[n]['fcoords']-G.nodes[nn]['fcoords']))
        dist = np.array(dist)
        if np.min(dist) < 1e-3:
            dist[np.argmin(dist)] = 1000
        if np.min(dist) > 1:
            continue

        nn = list(G.nodes())[np.argmin(dist)]
        node_node = G.nodes[n]['fcoords']-G.nodes[nn]['fcoords']
        node_node = node_node/norm(node_node)
        node_node_vec.append(node_node)
        
    #check if all of the pair vector are parallel
    for i in range(0,len(node_node_vec)):
        for j in range(i+1,len(node_node_vec)):
            if norm(np.cross(node_node_vec[i],node_node_vec[j])) > 1e-3:
                print('not pillar stack')
                return False,None
    pillar_vec = np.abs(node_node_vec[0])
    print('pillar stack',pillar_vec)
    return True,pillar_vec


PILLAR,pillar_vec = check_if_pillarstack(G)
        


pillar stack [1. 0. 0.]


In [47]:

beginning_point = [0.0,0.0,0.0]
rms,rot,tran = superimpose([beginning_point,node_pillar_cvec],[beginning_point,pillar_vec])
pillar_oriented_nodexvec=np.dot(np.asarray(node_x_vecs),rot)
pillar_oriented_node_xcoords = np.dot(unit_cell,pillar_oriented_nodexvec.T).T
pillar_oriented_node_fcoords = np.dot(chain_node_fcoords,rot)
pillar_oriented_node_coords = np.dot(unit_cell,pillar_oriented_node_fcoords.T).T
nodexxxx = []
xxxx_positions_dict = {}
chain_node_positions_dict = {}
chain_node_positions = []
#for n in sorted_nodes:
#    nodexxxx.append(G.nodes[n]['ccoords']+pillar_oriented_nodexvec)
#    chain_node_positions.append(G.nodes[n]['ccoords']+pillar_oriented_nodecoords)





In [48]:
# Add row indices as the first column
def addidx(array):
    row_indices = np.arange(array.shape[0]).reshape(-1, 1).astype(int)

    new_array = np.hstack((row_indices, array))
    return new_array

#reindex the nodes in the xxxx_positions with the index in the sorted_nodes, like G has 16 nodes[2,5,7], but the new dictionary should be [0,1,2]
xxxx_positions_dict = {sorted_nodes.index(n):addidx(G.nodes[n]['ccoords']+pillar_oriented_node_xcoords) for n in sorted_nodes}
chain_node_positions_dict = {sorted_nodes.index(n):G.nodes[n]['ccoords']+pillar_oriented_node_coords for n in sorted_nodes}
#reindex the edges in the G with the index in the sorted_nodes
sorted_edges = [(sorted_nodes.index(e[0]),sorted_nodes.index(e[1])) for e in sorted_edges]


In [49]:
len(nodexxxx),G.number_of_nodes(),G.nodes(),sorted_edges

(0,
 16,
 NodeView((1, 72, 2, 19, 3, 5, 76, 84, 14, 92, 0, 53, 37, 4, 61, 41)),
 [(1, 2),
  (0, 6),
  (0, 9),
  (1, 7),
  (2, 10),
  (3, 12),
  (3, 13),
  (4, 8),
  (4, 11),
  (5, 14),
  (5, 15)])

In [50]:
import numpy as np
from scipy.optimize import minimize, linear_sum_assignment
import networkx as nx


def reorthogonalize_matrix(matrix):
    """
    Ensure the matrix is a valid rotation matrix with determinant = 1.
    """
    U, _, Vt = np.linalg.svd(matrix)
    R = np.dot(U, Vt)
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = np.dot(U, Vt)
    return R

def find_optimal_pairings(node_i_positions, node_j_positions):
    """
    Find the optimal one-to-one pairing between atoms in two nodes using the Hungarian algorithm.
    """
    num_i, num_j = len(node_i_positions), len(node_j_positions)
    cost_matrix = np.zeros((num_i, num_j))
    for i in range(num_i):
        for j in range(num_j):
            cost_matrix[i, j] = np.linalg.norm(node_i_positions[i,1:] - node_j_positions[j,1:])

    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    print(list(zip(row_ind, col_ind)))
    return list(zip(row_ind, col_ind))

#pairing the atoms in two nodes, but with NOSYMNO as TRUE, 
# pairs of actual index are limit to [0,0],[1,1]...,the index is stored in the first column 
# of the atom_positions,we need to compare the distance between index pairs in both nodes, to find the optimal pairings
# the optimal pairings should have the minimum distance between the pair of atoms in two nodes
# the optimal pairings should be the same as the pair of index in the atom_positions
def find_nosymno_optimal_pairings(node_i_positions, node_j_positions,static_node_i_positions,static_node_j_positions):
    #find the optimal pair with same index in both nodes
    #index is stored in the first column of the atom_positions
    indices_of_node_i = node_i_positions[:,0]
    indices_of_node_j = node_j_positions[:,0]
    #find overlap index
    overlap_indices = np.intersect1d(indices_of_node_i, indices_of_node_j)
    #find the shortest distance between the overlap index
    distances = []
    for idx in overlap_indices:
        i = np.where(indices_of_node_i == idx)[0][0]
        j = np.where(indices_of_node_j == idx)[0][0]
        distances.append(np.linalg.norm(node_i_positions[i,1:] - node_j_positions[j,1:]))
    #find the index of the shortest distance
    sorted_indices = np.argsort(distances)
    
    #if more than one shortest distance, return the one in node_i who is closer to the middle point of mass center of node_ and node_j
    #if shorstest distance is more than one, find the center of mass of the overlap index, and find the distance between the center of mass and the overlap index
    #if the distance is larger, then switch the order of the index
    if len(sorted_indices)>1 :
        if (distances[sorted_indices[0]]-distances[sorted_indices[1]] < 1e-3):
            com_i = np.mean(static_node_i_positions[:,1:], axis=0)
            com_j = np.mean(static_node_j_positions[:,1:], axis=0)
            middle = (com_i + com_j) / 2
            def distance_to_middle(middle,idx_in_node_i):
                position_in_node_i = node_i_positions[np.where(indices_of_node_i == idx_in_node_i)[0][0],1:]
                return np.linalg.norm(middle - position_in_node_i)
            #if this egde is E type, i,j nodes are V type node: #this edge is inside,we should make the beginning towards inside 
            if distance_to_middle(middle,overlap_indices[sorted_indices[0]]) - distance_to_middle(middle,overlap_indices[sorted_indices[1]]) >1e-3:
                #switch the order of first and second index, others are the same
                new_sorted_indices = [sorted_indices[1],sorted_indices[0]]+list(sorted_indices[2:])
                sorted_ovelap_indices = [overlap_indices[idx] for idx in new_sorted_indices]
                return list(zip(np.where(indices_of_node_i == sorted_ovelap_indices[0])[0],np.where(indices_of_node_j == sorted_ovelap_indices[0])[0]))
            ##if this egde is DE type, one of i,j nodes are DV type node: #this edge is outside, we should make the beginning towards outside
            ##    if distance_to_middle(middle,overlap_indices[sorted_indices[0]]) - distance_to_middle(middle,overlap_indices[sorted_indices[1]]) >1e-3:
            ##        #switch the order of first and second index, others are the same
            ##        new_sorted_indices = [sorted_indices[1],sorted_indices[0]]+list(sorted_indices[2:])
            ##        sorted_ovelap_indices = [overlap_indices[idx] for idx in new_sorted_indices]
            ##        return list(zip(np.where(indices_of_node_i == sorted_ovelap_indices[0])[0],np.where(indices_of_node_j == sorted_ovelap_indices[0])[0]))

    #find the pairs of the overlap index, sort index by the distance, and return list(zip(overlap_indices,overlap_indices))
    #return a list zip of the index of the overlap index row index in node_i_positions and node_j_positions,  which is the optimal pair, order is the distance between opverlap index
    sorted_ovelap_indices = [overlap_indices[idx] for idx in sorted_indices]
    return list(zip(np.where(indices_of_node_i == sorted_ovelap_indices[0])[0],np.where(indices_of_node_j == sorted_ovelap_indices[0])[0]))

def update_pairs(pairs,atom_positions,i,j):
    nodeA_idx_set = atom_positions[i][:,0]
    nodeB_idx_set = atom_positions[j][:,0]
    correct_idx_pair =[]
    for k in range(len(pairs)):
        idx_A,idx_B = nodeA_idx_set[pairs[k][0]],nodeB_idx_set[pairs[k][1]]
        correct_idx_pair.append((idx_A,idx_B))
    return correct_idx_pair

def find_edge_pairings(sorted_edges, atom_positions,static_atom_positions):
    """
    Identify optimal pairings for each edge in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        dict: Mapping of edges to optimal atom pairs.
              Example: {(0, 1): [(0, 3), (1, 2)], ...}
    """

    edge_pairings = {}
    
    for i, j in sorted_edges:
        node_i_positions = atom_positions[i] #[index,x,y,z]
        node_j_positions = atom_positions[j] #[index,x,y,z]
        static_node_i_positions = static_atom_positions[i] #[index,x,y,z]
        static_node_j_positions = static_atom_positions[j] #[index,x,y,z]

        # Find optimal pairings for this edge
        #pairs = find_nosymno_optimal_pairings(node_i_positions, node_j_positions,static_node_i_positions,static_node_j_positions)
        pairs = find_optimal_pairings(node_i_positions, node_j_positions)
        idx_0,idx_1 = pairs[0]
       ##if i==1 and j==2: #TODO: TEST DEBUG
       ##    idx_0 = 1
       ##    idx_1 = 1
       ##    edge_pairings[(i, j)] = [(1,1)]
       ##    atom_positions[i] = np.delete(atom_positions[i], idx_0, axis=0)
       ##    atom_positions[j] = np.delete(atom_positions[j], idx_1, axis=0)
       ##    continue
        #if i==2 and j==1: #TODO: TEST DEBUG
        #    idx_0 = 1
        #    idx_1 = 1
        #    edge_pairings[(i, j)] = [(1,1)]
        #    atom_positions[i] = np.delete(atom_positions[i], idx_0, axis=0)
        #    atom_positions[j] = np.delete(atom_positions[j], idx_1, axis=0)
        #    continue
        x_idx_0 = atom_positions[i][idx_0][0]
        x_idx_1 = atom_positions[j][idx_1][0]
 
        edge_pairings[(i, j)] = update_pairs(pairs,atom_positions,i,j) #but only first pair match
        atom_positions[i] = np.delete(atom_positions[i], idx_0, axis=0)
        atom_positions[j] = np.delete(atom_positions[j], idx_1, axis=0)

    return edge_pairings


def axis_rotation_matrix(axis, theta):
    """
    Compute the rotation matrix for a rotation around an axis by an angle theta.

    Parameters:
        axis (tuple): The axis vector (a, b, c).
        theta (float): The rotation angle in radians.

    Returns:
        numpy.ndarray: The 3x3 rotation matrix.
    """
    a, b, c = axis
    axis = np.array([a, b, c])
    axis = axis / np.linalg.norm(axis)  # Normalize the axis vector
    a, b, c = axis

    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    I = np.eye(3)
    K = np.array([
        [0, -c, b],
        [c, 0, -a],
        [-b, a, 0]
    ])
    
    R = I + sin_theta * K + (1 - cos_theta) * np.dot(K, K)
    return R

def axis_objective_function(thetas, axis, static_atom_positions, edge_pairings):
    """
    Objective function to minimize distances between paired atoms along edges.

    Parameters:
        theta (float): The rotation angle in radians.
        axis (tuple): The axis vector (a, b, c).
        G (networkx.Graph): Graph structure.
        static_atom_positions (dict): Original positions of X atoms for each node.
        edge_pairings (dict): Precomputed pairings for each edge.

    Returns:
        float: Total distance metric to minimize.
    """
    total_distance = 0.0

    for (i, j), pairs in edge_pairings.items():
        R_i = reorthogonalize_matrix(axis_rotation_matrix(axis, thetas[i]))
        R_j = reorthogonalize_matrix(axis_rotation_matrix(axis, thetas[j]))

        com_i = G.nodes[sorted_nodes[i]]['ccoords']
        com_j = G.nodes[sorted_nodes[j]]['ccoords']

        # Rotate positions around their mass center
        rotated_i_positions = np.dot(static_atom_positions[i][:,1:] - com_i, R_i.T) + com_i
        rotated_j_positions = np.dot(static_atom_positions[j][:,1:] - com_j, R_j.T) + com_j

        for idx_i, idx_j in pairs:
            dist = np.linalg.norm(rotated_i_positions[int(idx_i)] - rotated_j_positions[int(idx_j)])
            total_distance += dist ** 2

    return total_distance


def axis_optimize_rotations(axis, num_nodes,sorted_edges, atom_positions):
    """
    Optimize the rotation angles around a given axis to minimize the difference between
    rotated and target positions for each node.

    Parameters:
        axis (tuple): The axis vector (a, b, c).
        G (networkx.Graph): Graph structure.
        static_atom_positions (dict): Original positions of X atoms for each node.
        edge_pairings (dict): Precomputed pairings for each edge.
        initial_thetas (numpy.ndarray): Initial guesses for the rotation angles.

    Returns:
        numpy.ndarray: The optimized rotation angles for each node.
    """
    initial_thetas = np.zeros(num_nodes) # Initial guess for rotation angles
    static_atom_positions = atom_positions.copy()
    # Precompute edge-specific pairings
    edge_pairings = find_edge_pairings(sorted_edges, atom_positions,static_atom_positions)
    result = minimize(axis_objective_function, initial_thetas, 
                      args=(axis,  static_atom_positions, edge_pairings), 
                      method='L-BFGS-B',options={"maxiter": 5000, "disp": True})
    optimized_thetas = result.x

    # Compute the rotation matrices for each node
    optimized_rotations = [reorthogonalize_matrix(axis_rotation_matrix(axis, theta)) for theta in optimized_thetas]
    
    # Print the optimized pairings after optimization
    print("Optimized Pairings (after optimization):")
    for (i, j), pairs in edge_pairings.items():
        print(f"Node {i} and Node {j}:")
        for idx_i, idx_j in pairs:
            print(f"  node{i}_{int(idx_i)} -- node{j}_{int(idx_j)}")
    print()

    return optimized_rotations

def compute_rotation_with_pairing(connected_nodes, atom_positions,current_rotation_matrix):
    """
    Compute the optimal rotation matrix for node pairs, starting from the current rotation matrix.
    
    Parameters:
        node_i_positions (numpy.ndarray): Positions of X atoms in node i (Nx3 array).
        node_j_positions (numpy.ndarray): Positions of X atoms in node j (Mx3 array).
        current_rotation_matrix (numpy.ndarray): The current 3x3 rotation matrix for node i.

    Returns:
        rotation_matrix (numpy.ndarray): Optimized 3x3 rotation matrix for node i.
    """
    
    i, j = connected_nodes
    # Extract paired positions
    paired_node_i = atom_positions[i][:,1:]
    paired_node_j = atom_positions[j][:,1:]

    # Compute the centers of mass for both sets
    com_i = np.mean(paired_node_i, axis=0)
    com_j = np.mean(paired_node_j, axis=0)

    # Translate positions to the center of mass
    translated_i = paired_node_i - com_i
    translated_j = paired_node_j - com_j

    # Apply the current rotation matrix to node_i's positions
    rotated_translated_i = np.dot(translated_i, current_rotation_matrix.T)

    # Compute covariance matrix and SVD
    H = np.dot(rotated_translated_i.T, translated_j)
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(U, Vt)

    # Ensure the resulting matrix is a valid rotation matrix (det = 1)
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = np.dot(U, Vt)

    # Update the rotation matrix by combining the current and incremental rotation
    optimized_rotation_matrix = np.dot(R, current_rotation_matrix)

    return optimized_rotation_matrix

def objective_function(params, G, static_atom_positions, edge_pairings):
    """
    Objective function to minimize distances between paired atoms along edges.

    Parameters:
        params (numpy.ndarray): Flattened array of rotation matrices.
        G (networkx.Graph): Graph structure.
        atom_positions (dict): Original positions of X atoms for each node.
        edge_pairings (dict): Precomputed pairings for each edge.

    Returns:
        float: Total distance metric to minimize.
    """
    num_nodes = len(G.nodes())
    rotation_matrices = params.reshape(num_nodes, 3, 3)
    total_distance = 0.0

    for (i, j), pairs in edge_pairings.items():
        R_i = reorthogonalize_matrix(rotation_matrices[i])
        R_j = reorthogonalize_matrix(rotation_matrices[j])

        com_i = G.nodes[sorted_nodes[i]]['ccoords']
        com_j = G.nodes[sorted_nodes[j]]['ccoords']

        # Rotate positions around their mass center
        rotated_i_positions = np.dot(static_atom_positions[i][:,1:] - com_i, R_i.T) + com_i
        rotated_j_positions = np.dot(static_atom_positions[j][:,1:] - com_j, R_j.T) + com_j


        for idx_i, idx_j in pairs:
            dist = np.linalg.norm(rotated_i_positions[int(idx_i)] - rotated_j_positions[int(idx_j)])
            total_distance += dist ** 2

    return total_distance


def optimize_rotations(num_nodes, sorted_edges,atom_positions):
    """
    Optimize rotations for all nodes in the graph.

    Parameters:
        G (networkx.Graph): Graph structure with edges between nodes.
        atom_positions (dict): Positions of X atoms for each node.

    Returns:
        list: Optimized rotation matrices for all nodes.
    """
    initial_rotations = np.tile(np.eye(3), (num_nodes, 1)).flatten()
    static_atom_positions = atom_positions.copy()
    # Precompute edge-specific pairings
    edge_pairings = find_edge_pairings(sorted_edges, atom_positions,static_atom_positions)

    result = minimize(
        objective_function,
        initial_rotations,
        args=(G, static_atom_positions, edge_pairings),
        method="L-BFGS-B",
        options={"maxiter": 5000, "disp": True, 'ftol': 1e-9,  # Smaller tolerance on function value
                'gtol': 1e-5,  # Smaller gradient tolerance
                'maxfun': 10000 } # Higher max evaluations
    )

    

    optimized_rotations = result.x.reshape(num_nodes, 3, 3)
    optimized_rotations = [reorthogonalize_matrix(R) for R in optimized_rotations]
    
    # Print the optimized pairings after optimization
    print("Optimized Pairings (after optimization):")
    for (i, j), pairs in edge_pairings.items():
        print(f"Node {i} and Node {j}:")
        for idx_i, idx_j in pairs:
            print(f"  node{i}_{idx_i} -- node{j}_{idx_j}")
    print()

    return optimized_rotations,static_atom_positions

def apply_rotations_to_atom_positions(optimized_rotations, G, atom_positions):
    """
    Apply the optimized rotation matrices to the atom positions.

    Parameters:
        optimized_rotations (list): Optimized rotation matrices for each node.
        G (networkx.Graph): Graph structure.
        atom_positions (dict): Original positions of X atoms for each node.

    Returns:
        dict: Rotated positions for each node.
    """
    rotated_positions = {}

    for i, node in enumerate(sorted_nodes):
        #if node type is V
        #if 'DV' in G.nodes[node]['type']:
          #  continue
        R = optimized_rotations[i]
        original_positions = atom_positions[i]
        com = G.nodes[node]['ccoords']

        # Translate, rotate, and translate back to preserve the mass center
        translated_positions = original_positions - com
        rotated_translated_positions = np.dot(translated_positions, R.T)
        rotated_positions[node] = rotated_translated_positions + com

    return rotated_positions

def save_xyz(filename, rotated_positions):
    """
    Save the rotated positions to an XYZ file for visualization.
    """
    with open(filename, "w") as file:
        num_atoms = sum(len(positions) for positions in rotated_positions.values())
        file.write(f"{num_atoms}\n")
        file.write("Optimized structure\n")

        for node, positions in rotated_positions.items():
            for pos in positions:
                file.write(f"X{node}   {pos[0]:.8f} {pos[1]:.8f} {pos[2]:.8f}\n")



# Optimize rotations
num_nodes = G.number_of_nodes()

###3D free rotation
#optimized_rotations,static_xxxx_positions = optimize_rotations(num_nodes,sorted_edges, xxxx_positions_dict)


###2D axis rotation
axis = pillar_vec  # Rotate around x-axis
optimized_rotations = axis_optimize_rotations(axis, num_nodes, sorted_edges, xxxx_positions_dict)



# Apply rotations
rotated_positions = apply_rotations_to_atom_positions(optimized_rotations, G, chain_node_positions_dict)

# Save results to XYZ
save_xyz("optimized_xxxxstructure.xyz", rotated_positions)


[(0, 0), (1, 1)]
[(0, 0), (1, 1)]
[(0, 0)]
[(0, 1)]
[(0, 0)]
[(0, 0), (1, 1)]
[(0, 0)]
[(0, 0), (1, 1)]
[(0, 0)]
[(0, 0), (1, 1)]
[(0, 0)]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           16     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.51399D+03    |proj g|=  6.77230D+01

At iterate    1    f=  1.38192D+03    |proj g|=  4.86643D+01

At iterate    2    f=  1.24750D+03    |proj g|=  3.00794D+01

At iterate    3    f=  1.21359D+03    |proj g|=  1.27907D+01

At iterate    4    f=  1.20590D+03    |proj g|=  1.18758D+01

At iterate    5    f=  1.19990D+03    |proj g|=  7.86717D+00

At iterate    6    f=  1.19723D+03    |proj g|=  5.26691D+00

At iterate    7    f=  1.19502D+03    |proj g|=  4.85966D+00

At iterate    8    f=  1.17485D+03    |proj g|=  2.87587D+01


 This problem is unconstrained.



At iterate    9    f=  1.17320D+03    |proj g|=  3.10304D+01

At iterate   10    f=  1.15801D+03    |proj g|=  3.31398D+01

At iterate   11    f=  1.13528D+03    |proj g|=  1.41585D+01

At iterate   12    f=  1.13015D+03    |proj g|=  9.43376D+00

At iterate   13    f=  1.12862D+03    |proj g|=  2.92584D+00

At iterate   14    f=  1.12844D+03    |proj g|=  2.09513D+00

At iterate   15    f=  1.12835D+03    |proj g|=  8.75411D-01

At iterate   16    f=  1.12833D+03    |proj g|=  1.85855D-01

At iterate   17    f=  1.12833D+03    |proj g|=  3.18096D-02
Optimized Pairings (after optimization):
Node 1 and Node 2:
  node1_0 -- node2_0
  node1_1 -- node2_1
Node 0 and Node 6:
  node0_0 -- node6_0
  node0_1 -- node6_1
Node 0 and Node 9:
  node0_1 -- node9_0
Node 1 and Node 7:
  node1_1 -- node7_1
Node 2 and Node 10:
  node2_1 -- node10_0
Node 3 and Node 12:
  node3_0 -- node12_0
  node3_1 -- node12_1
Node 3 and Node 13:
  node3_1 -- node13_0
Node 4 and Node 8:
  node4_0 -- node8_0
  node4_1 -

In [51]:
#test for scale 
import numpy as np
from scipy.optimize import minimize

def unit_cell_to_cartesian_matrix(aL,bL,cL,alpha,beta,gamma):
    pi = np.pi
    """Convert unit cell parameters to a Cartesian transformation matrix."""
    aL,bL,cL,alpha,beta,gamma = list(map(float, (aL,bL,cL,alpha,beta,gamma)))
    ax = aL
    ay = 0.0
    az = 0.0
    bx = bL * np.cos(gamma * pi / 180.0)
    by = bL * np.sin(gamma * pi / 180.0)
    bz = 0.0
    cx = cL * np.cos(beta * pi / 180.0)
    cy = (cL * bL * np.cos(alpha * pi /180.0) - bx * cx) / by
    cz = (cL ** 2.0 - cx ** 2.0 - cy ** 2.0) ** 0.5
    unit_cell = np.asarray([[ax,ay,az],[bx,by,bz],[cx,cy,cz]]).T
    return unit_cell

def fractional_to_cartesian(fractional_coords, T):
    """Convert fractional coordinates to Cartesian using the transformation matrix."""
    return np.dot(fractional_coords, T.T)

def cartesian_to_fractional(cartesian_coords, unit_cell_inv):
    """Convert Cartesian coordinates to fractional coordinates using the inverse transformation matrix."""
    return np.dot(cartesian_coords, unit_cell_inv.T)

def objective_function(params, old_cell_params, old_cartesian_coords, new_cartesian_coords):
    a_new, b_new, c_new, alpha_new, beta_new, gamma_new = params
    a_old, b_old, c_old, alpha_old, beta_old, gamma_old = old_cell_params

    # Compute transformation matrix for the old unit cell
    T_old = unit_cell_to_cartesian_matrix(a_old, b_old, c_old, alpha_old, beta_old, gamma_old)
    T_old_inv= np.linalg.inv(T_old)
    #backup
    old_fractional_coords = cartesian_to_fractional(old_cartesian_coords, T_old_inv)

    # Compute transformation matrix for the new unit cell
    T_new = unit_cell_to_cartesian_matrix(a_new, b_new, c_new, alpha_new, beta_new, gamma_new)
    T_new_inv= np.linalg.inv(T_new)

    # Convert the new Cartesian coordinates to fractional coordinate using the old unit cell
    fractional_coords_new = cartesian_to_fractional(new_cartesian_coords, T_new_inv)

    # Minimize the difference between the calculated new Cartesian coordinates and the provided new Cartesian coordinates
    return np.sum(np.linalg.norm(fractional_coords_new - old_fractional_coords, axis=1)**2)

# Example usage
# Old cell parameters (example values)
old_cell_params = [a, b, c, ang_alpha, ang_beta, ang_gamma]  # [a, b, c, alpha, beta, gamma]

# Old Cartesian coordinates of points (example values)
old_cartesian_coords = np.array([
    [0.0, 0.0, 0.0],    # Point 1
    [2.5, 3.0, 3.5],    # Point 2
    [1.25, 1.5, 5.25],  # Point 3
    [0.5, 2.4, 4.9]     # Point 4
])


# New Cartesian coordinates of the same points (example values)
new_cartesian_coords = np.array([
    [0.0, 0.0, 0.0],    # Point 1
    [5, 6, 7],    # Point 2
    [2.5, 3, 10.5],    # Point 3
    [1, 4.8, 9.8]     # Point 4
])

# Initial guess for new unit cell parameters (e.g., slightly modified cell)
initial_params = [4.5, 4.5, 4.5, 90.0, 90.0, 90.0]

# Bounds: a, b, c > 0; angles [0, 180]
bounds = [(0, None)] * 3 + [(20, 180)] * 3

# Optimize using L-BFGS-B to minimize the objective function
result = minimize(
    objective_function,
    x0=initial_params,
    args=(old_cell_params, old_cartesian_coords, new_cartesian_coords),
    method='L-BFGS-B',
    bounds=bounds
)

# Extract optimized parameters
optimized_params = np.round(result.x,5)
print("Optimized New Cell Parameters:", optimized_params)


NameError: name 'a' is not defined

In [ ]:
#绕轴旋转失去的自由度,如果两个node是沿轴相邻,则第二个应该施加一个偏转矩阵,180度偏转为flip,3个cluster则旋转60相邻,以轴相邻组合的个数为变量决定偏转角度,这两个node合起来的质心应该在旋转轴上或者很近的距离
#如果两个node是沿轴不相邻,则不考虑
